In [2]:
# from fastapi import FastAPI
# from starlette.responses import JSONResponse
import joblib
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

In [3]:
class PytorchMultiClass(nn.Module):
    def __init__(self, num_features):
        super(PytorchMultiClass, self).__init__()

        self.layer_1 = nn.Linear(num_features, 128)
        self.layer_out = nn.Linear(128, 104)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.dropout(F.relu(self.layer_1(x)), training=self.training)
        x = self.layer_out(x)
        return self.softmax(x)

In [4]:
nn_model = PytorchMultiClass(4)
nn_model.load_state_dict(torch.load('../models/pt_base_state_dict'))

<All keys matched successfully>

In [6]:
nn_model._modules

OrderedDict([('layer_1', Linear(in_features=4, out_features=128, bias=True)),
             ('layer_out',
              Linear(in_features=128, out_features=104, bias=True)),
             ('softmax', Softmax(dim=1))])

In [7]:
[print(i) for i in nn_model._modules]

layer_1
layer_out
softmax


[None, None, None]

In [45]:
sc = joblib.load('../models/standard_scaler.joblib')

In [44]:
enc = joblib.load('../models/output_encoder.joblib')

In [51]:
obs = {
        'review_aroma': [33],
        'review_appearance': [18],
        'review_palate': [46],
        'review_taste': [1]
    }

In [52]:
obsc = sc.transform(pd.DataFrame(obs))
obst = torch.from_numpy(obsc).float()

In [53]:
pred = nn_model(obst)

In [54]:
pred = pred.argmax(1).tolist()

In [56]:
enc.inverse_transform(pred)

array(['American Double / Imperial IPA'], dtype=object)

In [57]:
enc.transform(["American Double / Imperial IPA"])

array([9])